<a href="https://colab.research.google.com/github/Abidt2002/AI-Restaurant-System/blob/main/AI_Powered_Restaurant_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install nbformat nbconvert nbstripout --quiet

import nbformat
from nbconvert.preprocessors import ClearOutputPreprocessor

# Path to your AI Powered Resturant System
notebook_path = '/content/drive/MyDrive/Colab Notebooks/AI Powered Restaurant System.ipynb'

# Load the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Clear outputs
clear_output = ClearOutputPreprocessor()
nb, _ = clear_output.preprocess(nb, {})

# Remove widgets metadata if exists
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Save cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"✅ Notebook '{notebook_path}' is now GitHub-safe!")


✅ Notebook '/content/drive/MyDrive/Colab Notebooks/AI Powered Restaurant System.ipynb' is now GitHub-safe!


In [2]:
# Cell A: Install deps
!pip install -q chromadb sentence-transformers transformers gradio bcrypt pypdf
print("✅ Installed dependencies")


✅ Installed dependencies


In [3]:
# Cell B: Imports & basic config
import os, io, uuid, sqlite3, datetime, re
import bcrypt
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr
import pandas as pd

# Paths
DB_FILE = "restaurant.db"
CHROMA_DIR = "./chroma_store"   # change to drive path if mounting Drive

# Globals
CURRENT_USER = None   # single-user session for demo
print("✅ Config ready")


✅ Config ready


In [4]:
import sqlite3
conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Add 'role' column if missing
try:
    cur.execute("ALTER TABLE users ADD COLUMN role TEXT DEFAULT 'customer'")
    conn.commit()
    print("✅ Added 'role' column to users table")
except Exception as e:
    print("⚠️ Column might already exist:", e)


⚠️ Column might already exist: duplicate column name: role


In [5]:
import sqlite3

conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Create users table with role column
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password_hash TEXT,
    role TEXT DEFAULT 'customer'
)
""")
conn.commit()
print("✅ Users table created successfully")


✅ Users table created successfully


In [6]:
import bcrypt

# Insert default admin if not exists
cur.execute("SELECT COUNT(*) FROM users WHERE username=?", ("admin",))
if cur.fetchone()[0] == 0:
    pw = bcrypt.hashpw("admin123".encode(), bcrypt.gensalt())
    cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                ("admin", pw, "admin"))
    conn.commit()
    print("✅ Default admin created (username: admin, password: admin123)")
else:
    print("ℹ️ Admin already exists")

conn.close()


ℹ️ Admin already exists


In [7]:
import sqlite3, bcrypt

# Connect (or reopen if previously closed)
conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Create users table
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password_hash TEXT,
    role TEXT DEFAULT 'customer'
)
""")

# Insert default admin if not exists
cur.execute("SELECT COUNT(*) FROM users WHERE username=?", ("admin",))
if cur.fetchone()[0] == 0:
    pw = bcrypt.hashpw("admin123".encode(), bcrypt.gensalt())
    cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                ("admin", pw, "admin"))
    conn.commit()
    print("✅ Default admin created (username: admin, password: admin123)")
else:
    print("ℹ️ Admin already exists")

# Close connection after all operations
conn.close()


ℹ️ Admin already exists


In [8]:
# Cell D: Initialize SQLite DB and tables
conn = sqlite3.connect(DB_FILE, check_same_thread=False)
cur = conn.cursor()

# Users table (with roles)
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password_hash BLOB,
    role TEXT DEFAULT 'customer'
);
""")

# Operational tables
cur.execute("""
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT,
    item_name TEXT,
    quantity INTEGER,
    status TEXT DEFAULT 'Pending',
    created_at TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS reservations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT,
    table_number INTEGER,
    dt TEXT,
    status TEXT DEFAULT 'Booked',
    created_at TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    item_name TEXT UNIQUE,
    stock INTEGER DEFAULT 0
);
""")
conn.commit()

# Create demo admin if no users exist
cur.execute("SELECT COUNT(1) FROM users")
if cur.fetchone()[0] == 0:
    pw = bcrypt.hashpw("admin123".encode(), bcrypt.gensalt())
    cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                ("admin", pw, "admin"))
    conn.commit()
    print("✅ Default admin created: username=admin password=admin123  — change it!")
else:
    print("✅ Database initialized (tables ready)")


✅ Database initialized (tables ready)


In [9]:
# Cell E: Authentication helpers
def register_user(username, password, role="customer"):
    cur.execute("SELECT 1 FROM users WHERE username=?", (username,))
    if cur.fetchone():
        return False, "Username already exists."
    pw_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    cur.execute("INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                (username, pw_hash, role))
    conn.commit()
    return True, f"User '{username}' created with role '{role}'."

def login_user(username, password):
    global CURRENT_USER
    cur.execute("SELECT id, password_hash, role FROM users WHERE username=?", (username,))
    r = cur.fetchone()
    if not r:
        return False, "User not found."
    uid, pw_hash, role = r
    if bcrypt.checkpw(password.encode(), pw_hash):
        CURRENT_USER = {"id": uid, "username": username, "role": role}
        return True, f"Logged in as {username} (role={role})"
    return False, "Incorrect password."

def logout_user():
    global CURRENT_USER
    CURRENT_USER = None
    return "Logged out."


In [10]:
# Cell F: Chroma client and embedding model
# Use persistent Chroma if available (duckdb+parquet) or default client
try:
    client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=CHROMA_DIR))
except Exception:
    client = chromadb.Client()
# create or get collection
COLL_NAME = "restaurant_knowledge"
try:
    collection = client.get_collection(COLL_NAME)
except Exception:
    collection = client.create_collection(name=COLL_NAME, metadata={"created": str(datetime.datetime.utcnow())})

# Embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ Chroma collection ready and embeddings loaded")


/tmp/ipython-input-795355989.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  collection = client.create_collection(name=COLL_NAME, metadata={"created": str(datetime.datetime.utcnow())})
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


✅ Chroma collection ready and embeddings loaded


In [11]:
# Cell G: Seed KB with example docs (replace with your own)
seed_docs = [
    "Margherita Pizza: Tomato, Mozzarella, Basil. Allergens: Dairy.",
    "Chicken Biryani: Rice, Chicken, spices. Not vegetarian.",
    "Caesar Salad: Lettuce, Parmesan, Croutons. Allergens: Dairy, Gluten.",
    "Masala Chai: Tea, Milk, Spices. Allergens: Dairy.",
    "Hours: We are open daily from 12:00 to 23:30.",
    "Refund policy: Contact within 24 hours for order issues."
]
metas = [{"source": f"seed_{i}"} for i in range(len(seed_docs))]

# chunking not necessary for very small docs — for bigger files chunk before ingest
embs = embed_model.encode(seed_docs, convert_to_numpy=True).tolist()
ids = [str(uuid.uuid4()) for _ in seed_docs]
collection.add(documents=seed_docs, metadatas=metas, ids=ids, embeddings=embs)
print(f"✅ Ingested {len(seed_docs)} seed docs into RAG KB")


✅ Ingested 6 seed docs into RAG KB


In [12]:
# Cell H: Initialize HF model (text2text)
# flan-t5-large is a good free tradeoff; use GPU if available (device=0).
# On CPU set device=-1. If you have Colab GPU, set device=0 and enable GPU runtime.
llm = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)

SYSTEM_INSTR = ("You are a concise Restaurant Assistant. Use only the provided context to answer menu, "
                "allergen and policy questions. If the context doesn't contain the answer, say you don't know.")

def retrieve_docs(query, top_k=3):
    q_emb = embed_model.encode([query], convert_to_numpy=True).tolist()
    res = collection.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas","distances","ids"])
    docs = res["documents"][0] if "documents" in res and res["documents"] else []
    metas = res["metadatas"][0] if "metadatas" in res and res["metadatas"] else []
    return docs, metas

def rag_answer(query, top_k=3):
    docs, metas = retrieve_docs(query, top_k=top_k)
    if not docs:
        return "I don't have relevant documents yet. Please ingest your menu or docs."
    context = "\n\n".join([f"[{m.get('source','unknown')}]\n{d}" for d,m in zip(docs, metas)])
    prompt = f"{SYSTEM_INSTR}\n\nCONTEXT:\n{context}\n\nQUESTION: {query}\n\nAnswer concisely:"
    out = llm(prompt, max_length=200, do_sample=False)[0]['generated_text']
    srcs = ", ".join(sorted({m.get("source","unknown") for m in metas}))
    return out + (f"\n\nSources: {srcs}" if srcs else "")


Device set to use cpu


In [13]:
# Cell I: operational functions
def add_order(customer_name, item_name, quantity):
    now = datetime.datetime.now().isoformat(sep=" ", timespec="seconds")
    cur.execute("INSERT INTO orders (customer_name, item_name, quantity, created_at) VALUES (?, ?, ?, ?)",
                (customer_name, item_name, int(quantity), now))
    conn.commit()
    return f"Order created: {customer_name} — {quantity}x {item_name}"

def list_orders(limit=50):
    return cur.execute("SELECT id, customer_name, item_name, quantity, status, created_at FROM orders ORDER BY id DESC LIMIT ?",
                       (limit,)).fetchall()

def update_order_status(order_id, new_status):
    cur.execute("UPDATE orders SET status=? WHERE id=?", (new_status, int(order_id)))
    conn.commit()
    return f"Order {order_id} status set to {new_status}"

def add_reservation(customer_name, table_number, dt_str):
    now = datetime.datetime.now().isoformat(sep=" ", timespec="seconds")
    cur.execute("INSERT INTO reservations (customer_name, table_number, dt, created_at) VALUES (?, ?, ?, ?)",
                (customer_name, int(table_number), dt_str, now))
    conn.commit()
    return f"Reservation confirmed for {customer_name} at table {table_number} on {dt_str}"

def list_reservations(limit=50):
    return cur.execute("SELECT id, customer_name, table_number, dt, status, created_at FROM reservations ORDER BY id DESC LIMIT ?",
                       (limit,)).fetchall()

def upsert_inventory(item_name, qty):
    cur.execute("SELECT id FROM inventory WHERE item_name=?", (item_name,))
    if cur.fetchone():
        cur.execute("UPDATE inventory SET stock=? WHERE item_name=?", (int(qty), item_name))
    else:
        cur.execute("INSERT INTO inventory (item_name, stock) VALUES (?, ?)", (item_name, int(qty)))
    conn.commit()
    return f"Inventory set: {item_name} -> {qty}"

def get_inventory(item_name=None):
    if item_name:
        return cur.execute("SELECT item_name, stock FROM inventory WHERE item_name=?", (item_name,)).fetchone()
    return cur.execute("SELECT item_name, stock FROM inventory ORDER BY item_name").fetchall()


In [14]:
# Cell J: minimal NLU-like router for commands
def db_intent_handler(message):
    m = message.lower()
    if "orders" in m or "order list" in m:
        rows = list_orders()
        if not rows:
            return "No orders yet."
        return "\n".join([f"#{r[0]} | {r[1]} — {r[3]}x {r[2]} (Status: {r[4]})" for r in rows])
    if "reservations" in m or "bookings" in m:
        rows = list_reservations()
        if not rows:
            return "No reservations yet."
        return "\n".join([f"#{r[0]} | {r[1]} — Table {r[2]} at {r[3]} (Status: {r[4]})" for r in rows])
    if "inventory" in m or "stock" in m:
        mm = re.search(r"(?:stock of|inventory of|inventory|stock)\s*(?:for\s*)?([\w\s]+)?", m)
        if mm and mm.group(1) and mm.group(1).strip():
            item = mm.group(1).strip().title()
            row = get_inventory(item)
            return f"{row[0]}: {row[1]}" if row else f"No inventory item named '{item}'."
        rows = get_inventory()
        return "\n".join([f"{r[0]}: {r[1]}" for r in rows]) if rows else "Inventory empty."
    return None

def command_router(message):
    msg = message.strip()
    # order <item> <qty>  OR  order <customer> <item> <qty>
    if msg.lower().startswith("order "):
        parts = msg.split()
        if len(parts) == 3:
            item, qty = parts[1], parts[2]
            customer = CURRENT_USER['username'] if CURRENT_USER else "Guest"
            return add_order(customer, item, qty), True
        elif len(parts) >= 4:
            customer, item, qty = parts[1], parts[2], parts[3]
            return add_order(customer, item, qty), True
        return ("Format: order <item> <qty>  OR  order <customer> <item> <qty>", True)

    # reserve <name> <table> <datetime>
    if msg.lower().startswith("reserve "):
        parts = msg.split()
        if len(parts) >= 4:
            name = parts[1]
            try:
                table = int(parts[2])
            except:
                return ("Table number must be integer.", True)
            dt = " ".join(parts[3:])
            return add_reservation(name, table, dt), True
        return ("Format: reserve <name> <table> <YYYY-MM-DD HH:MM>", True)

    # update inventory <item> <qty>  (admin only)
    if msg.lower().startswith("update inventory "):
        if not CURRENT_USER or CURRENT_USER.get("role") != "admin":
            return ("Only admins can update inventory.", True)
        parts = msg.split()
        if len(parts) >= 4:
            item = parts[2].title()
            try:
                qty = int(parts[3])
            except:
                return ("Quantity must be integer.", True)
            return upsert_inventory(item, qty), True
        return ("Format: update inventory <item> <qty>", True)

    # set order <id> <Status>
    if msg.lower().startswith("set order "):
        if not CURRENT_USER or CURRENT_USER.get("role") not in ("admin","waiter"):
            return ("Only admin/waiter can update order status.", True)
        parts = msg.split()
        if len(parts) >= 4:
            oid, status = parts[2], " ".join(parts[3:])
            return update_order_status(oid, status), True
        return ("Format: set order <id> <new_status>", True)

    return (None, False)


In [15]:
# Cell K: ingestion helper (handles .txt, .md, .csv, .pdf)
from pypdf import PdfReader

def chunk_text(text, chunk_size=800, overlap=150):
    if len(text) <= chunk_size:
        return [text]
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

def ingest_uploaded_files(file_objs):
    all_texts = []
    all_meta = []
    for f in file_objs:
        name = f.name
        raw = f.read()
        try:
            if name.lower().endswith(".pdf"):
                reader = PdfReader(io.BytesIO(raw))
                text = "\n\n".join([p.extract_text() or "" for p in reader.pages])
            else:
                text = raw.decode("utf-8", errors="ignore")
        except Exception:
            text = raw.decode("utf-8", errors="ignore") if isinstance(raw, (bytes,bytearray)) else str(raw)
        chunks = chunk_text(text, 800, 150)
        for c in chunks:
            all_texts.append(c)
            all_meta.append({"source": name})
    if not all_texts:
        return "No text extracted."
    embs = embed_model.encode(all_texts, convert_to_numpy=True).tolist()
    ids = [str(uuid.uuid4()) for _ in all_texts]
    collection.add(documents=all_texts, metadatas=all_meta, ids=ids, embeddings=embs)
    return f"Ingested {len(all_texts)} chunks from {len(file_objs)} file(s)."


In [ ]:
# Cell L: Gradio UI (run at the end)
def login_ui(username, password):
    ok, msg = login_user(username, password)
    return msg

def register_ui(username, password, role):
    ok, msg = register_user(username, password, role)
    return msg

def chat_ui(history, message):
    # history: list of [user, assistant] pairs
    if not CURRENT_USER:
        return history + [[message, "⚠️ Please login first (Login tab)."]], ""
    # DB-intent
    db_res = db_intent_handler(message)
    if db_res:
        return history + [[message, db_res]], ""
    # command router
    resp, handled = command_router(message)
    if handled:
        return history + [[message, resp]], ""
    # fallback → RAG
    try:
        ans = rag_answer(message)
    except Exception as e:
        ans = f"[RAG error] {e}"
    return history + [[message, ans]], ""

def view_orders_ui():
    rows = list_orders()
    if not rows:
        return "No orders."
    df = pd.DataFrame(rows, columns=["id","customer","item","qty","status","created_at"])
    return df.to_markdown(index=False)

def view_reservations_ui():
    rows = list_reservations()
    if not rows:
        return "No reservations."
    df = pd.DataFrame(rows, columns=["id","customer","table","dt","status","created_at"])
    return df.to_markdown(index=False)

def view_inventory_ui():
    rows = get_inventory()
    if not rows:
        return "No inventory."
    df = pd.DataFrame(rows, columns=["item","stock"])
    return df.to_markdown(index=False)

with gr.Blocks(title="AI Restaurant (HuggingFace RAG + Auth + DB)") as demo:
    gr.Markdown("## 🍽️ AI-Powered Restaurant System — HuggingFace RAG + Secure Login")
    with gr.Tab("🔑 Login / Register"):
        with gr.Row():
            u = gr.Textbox(label="Username")
            p = gr.Textbox(label="Password", type="password")
            login_btn = gr.Button("Login")
            login_out = gr.Textbox(label="Status")
            login_btn.click(login_ui, [u, p], login_out)
        gr.Markdown("---")
        with gr.Row():
            ru = gr.Textbox(label="New username")
            rp = gr.Textbox(label="New password", type="password")
            rrole = gr.Dropdown(["admin","waiter","customer"], value="customer", label="Role")
            reg_btn = gr.Button("Register")
            reg_out = gr.Textbox(label="Register status")
            reg_btn.click(register_ui, [ru, rp, rrole], reg_out)
    with gr.Tab("💬 Assistant Chat"):
        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Ask or command: e.g., 'Which dishes are vegetarian?', 'order pizza 2'", lines=1)
        send = gr.Button("Send")
        send.click(chat_ui, [chatbot, msg], [chatbot, msg])
        msg.submit(chat_ui, [chatbot, msg], [chatbot, msg])
    with gr.Tab("📋 Dashboard"):
        with gr.Row():
            ord_btn = gr.Button("View Orders")
            ord_out = gr.Markdown()
            ord_btn.click(lambda: view_orders_ui(), None, ord_out)
            res_btn = gr.Button("View Reservations")
            res_out = gr.Markdown()
            res_btn.click(lambda: view_reservations_ui(), None, res_out)
            inv_btn = gr.Button("View Inventory")
            inv_out = gr.Markdown()
            inv_btn.click(lambda: view_inventory_ui(), None, inv_out)
        gr.Markdown("### Admin: Set Inventory")
        inv_item = gr.Textbox(label="Item")
        inv_qty = gr.Number(label="Stock")
        inv_up = gr.Button("Set Inventory")
        inv_up_out = gr.Textbox()
        def inv_up_ui(item, qty):
            if not CURRENT_USER or CURRENT_USER.get("role") != "admin":
                return "Only admin can update inventory."
            return upsert_inventory(item.title(), int(qty))
        inv_up.click(inv_up_ui, [inv_item, inv_qty], inv_up_out)
    with gr.Tab("📥 Ingest Docs"):
        uploader = gr.File(file_count="multiple", file_types=[".txt",".md",".csv",".pdf"])
        ingest_btn = gr.Button("Ingest")
        ingest_out = gr.Textbox()
        ingest_btn.click(ingest_uploaded_files, uploader, ingest_out)

# Launch the app (remove share=True in private use)
demo.launch(debug=True, share=True)


/tmp/ipython-input-2153749378.py:68: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://38ad730a680afdb50d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
